In [2]:
!pip install pandas dash plotly prophet

In [5]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import os

# Set up the Dash app
app = dash.Dash(__name__)

# Load the data from the Excel files
caminho = os.path.join('db_softtek' , 'cubo_anl')
path_arq = os.path.abspath(caminho)

# Load the necessary data
tbl_anl = pd.read_excel(os.path.join(path_arq, 'tbl_full_analitica.xlsx'))
cubo1_df = pd.read_excel(os.path.join(path_arq, 'cubo1_consolidado.xlsx'))
cubo2_df = pd.read_excel(os.path.join(path_arq, 'cubo2_consolidado.xlsx'))

# Example of preprocessing
tbl_anl['VALOR CONTRATO'] = tbl_anl['VALOR CONTRATO'].astype(float)

# Create the layout
app.layout = html.Div([
    html.H1("Dashboard de Demandas"),

    # Dropdowns for filtering
    dcc.Dropdown(
        id='anomes-dropdown',
        options=[{'label': anomes, 'value': anomes} for anomes in cubo1_df['ANOMES_REF'].unique()],
        value=cubo1_df['ANOMES_REF'].unique()[0]
    ),

    dcc.Dropdown(
        id='complexidade-dropdown',
        options=[{'label': comp, 'value': comp} for comp in cubo1_df['ATRIBUTO'].unique()],
        value=cubo1_df['ATRIBUTO'].unique()[0]
    ),

    # Graphs
    dcc.Graph(id='aberturas-encerramentos-graph'),
    dcc.Graph(id='encerrados-complexidade-graph'),
    dcc.Graph(id='cargo-custo-medio-graph'),
    dcc.Graph(id='leadtime-graph')
])

# Define callbacks for interactivity
@app.callback(
    Output('aberturas-encerramentos-graph', 'figure'),
    [Input('anomes-dropdown', 'value')]
)
def update_aberturas_encerramentos(selected_anomes):
    filtered_df = cubo1_df[cubo1_df['ANOMES_REF'] == selected_anomes]
    fig = px.bar(filtered_df, x='ATRIBUTO', y='CHAMADO', color='INDICADOR', barmode='group')
    return fig

@app.callback(
    Output('encerrados-complexidade-graph', 'figure'),
    [Input('anomes-dropdown', 'value')]
)
def update_encerrados_complexidade(selected_anomes):
    filtered_df = cubo1_df[(cubo1_df['ANOMES_REF'] == selected_anomes) & (cubo1_df['INDICADOR'] == 'ENCERRAMENTO')]
    fig = px.line(filtered_df, x='ATRIBUTO', y='CHAMADO', color='COMPLEXIDADE')
    return fig

@app.callback(
    Output('cargo-custo-medio-graph', 'figure'),
    [Input('anomes-dropdown', 'value')]
)
def update_cargo_custo_medio(selected_anomes):
    filtered_df = cubo2_df[cubo2_df['ANOMES_STATUS'] == selected_anomes]
    fig = px.bar(filtered_df, x='SENIORIDADE', y='HORAS', color='COMPLEXIDADE', barmode='group')
    return fig

@app.callback(
    Output('leadtime-graph', 'figure'),
    [Input('anomes-dropdown', 'value')]
)
def update_leadtime(selected_anomes):
    filtered_df = cubo1_df[(cubo1_df['ANOMES_REF'] == selected_anomes) & (cubo1_df['INDICADOR'] == 'LEADTIME')]
    fig = px.bar(filtered_df, x='ATRIBUTO', y='CHAMADO', color='LEADTIME_CALCULADO', barmode='group')
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
Cell In[5], line 63, in update_encerrados_complexidade(selected_anomes=202209)
     57 @app.callback(
     58     Output('encerrados-complexidade-graph', 'figure'),
     59     [Input('anomes-dropdown', 'value')]
     60 )
     61 def update_encerrados_complexidade(selected_anomes):
     62     filtered_df = cubo1_df[(cubo1_df['ANOMES_REF'] == selected_anomes) & (cubo1_df['INDICADOR'] == 'ENCERRAMENTO')]
---> 63     fig = px.line(filtered_df, x='ATRIBUTO', y='CHAMADO', color='COMPLEXIDADE')
        filtered_df = Empty DataFrame
Columns: [ANOMES_REF, ATRIBUTO2, ATRIBUTO, CHAMADO, INDICADOR, SENIORIDADE]
Index: []
        px = <module 'plotly.express' from '/workspaces/softek-2024/.venv/lib/python3.8/site-packages/plotly/express/__init__.py'>
     64     return fig

File /workspaces/softek-2024/.venv/lib/python3.8/site-packages/plotly/exp